In [1]:
import requests
import urllib.request
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [ ]:
chrome_opt = Options()
chrome_opt.headless = True
driver = webdriver.Chrome('driver/chromedriver',chrome_options=chrome_opt)

In [ ]:
driver.get(head_url)

In [ ]:
urls = driver.find_elements_by_class_name("nav-switch__next")

In [ ]:
ref = urls[0].get_attribute('data-href')
date = ref.split('date=')[1].split('%')[0]

In [ ]:
new_url = 'https://www.goal.com/en-us/premier-league/fixtures-results/' + date + '/2kwbbcootiqqgmrzs6o5inle5'
new_url

In [ ]:
driver.quit()

In [ ]:
# max_iter = 100
curr_url = 'https://www.goal.com/en-us/premier-league/fixtures-results/2021-01-02/2kwbbcootiqqgmrzs6o5inle5'
matches_url = list()
matches_url.append(curr_url)
driver.get(curr_url)
nxt_page = driver.find_elements_by_class_name("nav-switch__next")[0]
while nxt_page is not None:
    date = nxt_page.get_attribute('data-href').split('date=')[1].split('%')[0]
    nxt_url = 'https://www.goal.com/en-us/premier-league/fixtures-results/' + date + '/2kwbbcootiqqgmrzs6o5inle5'
    matches_url.append(nxt_url)
    driver.get(nxt_url)
    nxt_page = driver.find_elements_by_class_name("nav-switch__next")[0]

In [ ]:
"""
    Get url for match page
"""
match_prefix = 'https://www.goal.com'
match_pages = list()
for url in matches_url:
    driver.get(url)
    matches = driver.find_elements_by_class_name('match-main-data-link')
    for match in matches:
        href = match.get_attribute('href')
        splits = href.split('/')
        commentary_url = splits[0]+'//'+splits[1]+'/'+splits[2]+'/'+\
splits[3]+'/'+splits[4]+'/'+splits[5]+'/commentary-result/'+splits[6] 
        
        match_pages.append(commentary_url)

In [ ]:
with open('match_pages.txt', 'w') as f:
    for item in match_pages : f.write('%s\n' % item)

In [2]:
match_pages = None
with open('match_pages.txt', 'r') as f:
    match_pages = f.read().splitlines()
    print(len(match_pages))

3874


In [91]:
url = 'https://www.goal.com/en-us/match/brighton-hove-albion-v-wolverhampton-wanderers/8jhrtgfo31i2az79l5e6d2adw'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

In [34]:
event = soup.find_all('div', class_='comment-event')[0]
desc = soup.find_all('div', class_='comment-desc')[0]

In [35]:
event.text

'       Fábio Silva   Yellow Card       '

In [36]:
desc.text

'  It is a midweek trip to West Ham next up for the Baggies in the Premier League - but Wolves are back in FA Cup action, with a fourth round tie against Chorley. Until then however, thank you for joining us - and have a great weekend!  '

In [37]:
soup.select(".comment")[10].get_text()

"  84'  Gibbs has a clumsy nibble at Traore and concedes the free-kick around 35 yards out in midfield. The delivery is dangerous but Button manages to make a safe connection to collect.   "

In [38]:
soup.select(".comment .comment-event")[0].get_text()

'       Fábio Silva   Yellow Card       '

In [98]:
for page in match_pages:
    splits = page.split('/')
    url = splits[0] + '//'
    for s in splits[3:] : url += s + '/'
    

https://www.goal.com/en-us/match/tottenham-hotspur-v-leeds-united/commentary-result/8ka4lbdcciz7f38za3dwvjytw/
https://www.goal.com/en-us/match/crystal-palace-v-sheffield-united/commentary-result/8jrh8s08xe1ih3qztdvfdm710/
https://www.goal.com/en-us/match/brighton-hove-albion-v-wolverhampton-wanderers/commentary-result/8jhrtgfo31i2az79l5e6d2adw/
https://www.goal.com/en-us/match/west-bromwich-albion-v-arsenal/commentary-result/8kdba6nqqw2at21dxne4t0hlg/
https://www.goal.com/en-us/match/newcastle-united-v-leicester-city/commentary-result/8k3h8bepmyl3x0i91zk7n6rys/
https://www.goal.com/en-us/match/chelsea-v-manchester-city/commentary-result/8jobnl84s86y0zxac38tbgw7o/
https://www.goal.com/en-us/match/southampton-v-liverpool/commentary-result/8k6wtn5hn7whnxgbfvmkeuy38/
https://www.goal.com/en-us/match/sheffield-united-v-newcastle-united/commentary-result/8kyga41m1kxtb32eqzv5z5fro/
https://www.goal.com/en-us/match/burnley-v-manchester-united/commentary-result/85l57jh7jxgumy8faimtcxe6s/
https

In [3]:
from tqdm.notebook import tqdm

terms = ['Yellow Card', 'Penalty Goal', 'Assist', 'Goal', 'Red Card', 'Own Goal', 'Penalty Save',
         'Missed Penalty']
lst_time = list()
lst_player_sub = list()
lst_event_type = list()
lst_main_player = list()
lst_commentary = list()
for page in tqdm(match_pages) :
    splits = page.split('/')
    url = splits[0] + '//'
    for s in splits[3:] : url += s + '/'
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    event = soup.find_all('div', class_='comment')
    for e in event:
        try :
            ev = e.find('div', class_='comment-event').text.split()
            lst_main_player.append(ev[0] + ' ' + ev[1])
            rest = ''
            for r in ev[2:]: rest += r + ' '
            rest = rest.strip()
            if rest in terms : 
                lst_event_type.append(rest)
                lst_player_sub.append('na')
            else :
                lst_event_type.append('Substitution')
                lst_player_sub.append(rest)
                
            info = e.find('div', class_='comment-desc').text.split()
            time = info[0]
            start = 1
            if info[1] == '+' : 
                time += ' + ' + info[2]
                start = 3
            lst_time.append(time)
            commentary = ''
            for i in info[start:] : commentary += i + ' '
            commentary = commentary.strip()
            lst_commentary.append(commentary)

        except: 
            continue

  0%|          | 0/3874 [00:00<?, ?it/s]

In [4]:
len(lst_player_sub)

39427

In [5]:
import pandas as pd

df = pd.DataFrame({'time' : lst_time,
                   'event' : lst_event_type,
                   'player_sub' : lst_player_sub,
                   'main_player' : lst_main_player,
                   'commentary' : lst_commentary})
df.head()

,time,event,player_sub,main_player,commentary
0,89',Yellow Card,na,K. Phillips,A hasty challenge from Kalvin Phillips now and...
1,87',Substitution,H. Kane,Carlos Vinícius,Harry Kane - who became a father this week - m...
2,85',Yellow Card,na,P. Højbjerg,"Hojbjerg picks up a late booking here, as the ..."
3,78',Substitution,T. Ndombèlé,Lucas Moura,"Tottenham make their second change now, with N..."
4,76',Substitution,H. Winks,M. Sissoko,Lloris gets a glance to this one and Ayling's ...


In [6]:
df['event'].unique()

array(['Yellow Card', 'Substitution', 'Assist', 'Goal', 'Penalty Goal',
       'Own Goal', 'Red Card'], dtype=object)

In [7]:
df.to_pickle('event_data.pickle')

In [9]:
df.iloc[4,-1]

"Lloris gets a glance to this one and Ayling's return ball sails wide of the mark. Moussa Sissoko replaces Winks for the hosts."